## Initialization

In [ ]:
!mkdir -p /data/sets/nuscenes  # Make the directory to store the nuScenes dataset in.

!wget https://www.nuscenes.org/data/v1.0-mini.tgz  # Download the nuScenes mini split.

!tar -xf v1.0-mini.tgz -C /data/sets/nuscenes  # Uncompress the nuScenes mini split.

!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.

In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
import numpy as np

nusc = NuScenes(version='v1.0-trainval', dataroot='/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 88.140 seconds.
Reverse indexing ...
Done reverse indexing in 12.6 seconds.


In [2]:
from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.map_expansion import arcline_path_utils
from nuscenes.map_expansion.bitmap import BitMap

nusc_map = NuScenesMap(dataroot='/work/apperception/data/raw/nuScenes/Map-expansion', map_name='boston-seaport')

In [3]:
class Instance():
  def __init__(self, timestamp, sample_record, annotation_token, ego_pose_token, annotation_token2=None, annotation_token3=None):
    self.timestamp = timestamp
    self.sample_record = sample_record
    self.annotation_token = annotation_token
    self.annotation_token2 = annotation_token2
    self.annotation_token3 = annotation_token3
    self.ego_pose_token = ego_pose_token

In [4]:
from pyquaternion import Quaternion
import math

def normalizeAngle(angle) -> float:
    while angle > math.pi:
        angle -= math.tau
    while angle < -math.pi:
        angle += math.tau
    assert -math.pi <= angle <= math.pi
    return angle

def get_heading(rotation):
    yaw = Quaternion(rotation).yaw_pitch_roll[0]
    return normalizeAngle(yaw)

In [5]:
def get_road_direction(position):
  x, y, z = position
  closest_lane = nusc_map.get_closest_lane(x, y, radius=2)
  if closest_lane:
    lane_record = nusc_map.get_arcline_path(closest_lane)
    poses = arcline_path_utils.discretize_lane(lane_record, resolution_meters=1)
    closest_pose_on_lane, distance_along_lane = arcline_path_utils.project_pose_to_lane((x, y, 0), lane_record)
    yaw = closest_pose_on_lane[2]
    return normalizeAngle(yaw)
  else:
    return None  

In [6]:
def is_contained(position, segment_type):
  x, y, z = position
  return nusc_map.layers_on_point(x, y)[segment_type] != ''

In [7]:
def is_contained_intersection(position):
  x, y, z = position
  road_segment_token = nusc_map.layers_on_point(x, y)['road_segment']
  if road_segment_token != '':
    road_segment = nusc_map.get('road_segment', road_segment_token)
    return road_segment["is_intersection"]
  else:
    return False

In [8]:
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix

def is_visible(sample_record, annotation_token):
  camera = sample_record["data"]["CAM_FRONT"]
  _, boxes, _ = nusc.get_sample_data(camera, box_vis_level=BoxVisibility.ANY, selected_anntokens=[annotation_token])
  return len(boxes) > 0

In [9]:
def convert_camera(cam_position, cam_heading, obj_point):
    cam_x, cam_y, _ = cam_position
    obj_x, obj_y, _ = obj_point
    
    subtract_x = obj_x - cam_x
    subtract_y = obj_y - cam_y

    subtract_mag = np.sqrt(subtract_x**2 + subtract_y**2)

    res_x = subtract_mag * np.cos(-cam_heading + np.arctan2(subtract_y, subtract_x))
    res_y = subtract_mag * np.sin(-cam_heading + np.arctan2(subtract_y, subtract_x))

    return res_x, res_y

## Figure 12


In [ ]:
# world = world.filter(
#     F.like(o.type, lit('human.pedestrian%')) &
#     F.contained(c.ego, 'road') &
#     (F.contained_margin(o.bbox@c.time, F.road_segment('road'), lit(0.50)) | F.contained(o.trans@c.time, 'road')) &
#     F.angle_excluding(F.facing_relative(o.traj@c.time, c.ego), lit(-70), lit(70)) &
#     F.angle_between(F.facing_relative(c.ego, c.roadDirection), lit(-15), lit(15)) &
#     (F.distance(c.camAbs, o.traj@c.time) < lit(50)) &
#     (F.view_angle(o.trans@c.time, c.camAbs) < lit(35))
# )

In [ ]:
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

In [ ]:
### Devkit Plan
# 1. Get all instances of Egos
#     - such that on road (prob not needed)
#     - such that facing Range(-15, 15) relative to roadDirection
# 2. Geet all instances of pedestrians wrt egos
#     - such that withitn 50 meters of ego
#     - such that on road
#     - such that visibile from ego (viewAngle if needed)

In [ ]:
## Get all possible pedestrian + ego instances
instances = []
for sample in nusc.sample:
  scene = nusc.get('scene', sample['scene_token'])
  log = nusc.get('log', scene['log_token'])
  if log['location'] == 'boston-seaport':
    for annotation in sample['anns']:
      annotation_metadata = nusc.get('sample_annotation', annotation)
      if annotation_metadata['category_name'] == 'human.pedestrian.adult':
        CAM_FRONT_SENSOR = sample['data']['CAM_FRONT']
        sample_data = nusc.get('sample_data', CAM_FRONT_SENSOR)
        ego_pose_token = sample_data['ego_pose_token']
        instances.append(Instance(timestamp=sample_data['timestamp'], sample_record=sample, annotation_token=annotation, ego_pose_token=ego_pose_token))
len(instances)

In [ ]:
## Filter for pedestrians that are within 50 meters of the ego
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)

  ped_trans = np.array(annotation['translation'])
  ego_trans = np.array(ego_pose['translation'])
  distance = np.linalg.norm(ped_trans - ego_trans)

  if distance < 50:
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter so that pedestrian is visible from ego
new_instances = []
for instance in instances:
  ped_visible = is_visible(sample_record=instance.sample_record, annotation_token=instance.annotation_token)
  if ped_visible:
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter so that for pedestrians that are on a road
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)

  ped_trans = np.array(annotation['translation'])
  ego_trans = np.array(ego_pose['translation'])
  
  if is_contained(ped_trans, 'road_segment'):
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter for egos whos heading is aligned with the road direction
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)

  ped_trans = np.array(annotation['translation'])
  ego_trans = np.array(ego_pose['translation'])
  
  ego_heading = get_heading(ego_pose['rotation'])
  road_direction = get_road_direction(ego_trans)
  if road_direction == None:
    new_instances.append(instance)
  elif normalizeAngle(abs(ego_heading - road_direction)) < math.radians(15):
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## require abs(relative heading of ped from ego) > 70 deg
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)

  ped_trans = np.array(annotation['translation'])
  ego_trans = np.array(ego_pose['translation'])
  
  ego_heading = get_heading(ego_pose['rotation'])
  ped_heading = get_heading(annotation['rotation'])
  if abs(ego_heading - ped_heading) > math.radians(70):
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
results = {}
for instance in instances:
  camera_token = instance.sample_record['data']['CAM_FRONT']
  camera_data = nusc.get('sample_data', camera_token)
  results[camera_data['token']] = camera_data['filename'].split('/')[2]

In [ ]:
for token in results:
  nusc.render_sample_data(token)

In [ ]:
############# Validation ####################
expected_imgs = [
    'n008-2018-05-21-11-06-59-0400__CAM_FRONT__1526915471412465.jpg',
    'n008-2018-07-27-12-07-38-0400__CAM_FRONT__1532707917112404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153662404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154662404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385155162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385158662404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159162404.jpg',
    'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159662404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805162404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805662415.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806162404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806612404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807162404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807662404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656808162404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404362404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404762404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405262404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405762404.jpg',
    'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659406262404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535728830362404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326912404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327912404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328912404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329412404.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329912795.jpg',
    'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729330362404.jpg',
    'n008-2018-08-31-11-37-23-0400__CAM_FRONT__1535730293412404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236162404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236662404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237112404.jpg',
    'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237612404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291162404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291662404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292162404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292662404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293162404.jpg',
    'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293662404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299143862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148862404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299149412404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299229362404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299239112404.jpg',
    'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299240112404.jpg',
]

missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in results.values():
        missing += 1
        # missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
# show_images(missing_images, sample=10)

extra = 0
extra_images = {}
for img in results:
    if results[img] not in expected_imgs:
        extra += 1
        # extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(results), "=", 100 * extra / len(results), "%")
# show_images(extra_images.keys(), sample=10)

In [ ]:
fig, ax = nusc_map.render_layers(nusc_map.non_geometric_layers, figsize=1)

## Figure 13


In [ ]:
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

In [ ]:
# world = world.filter(
#     (obj1.id != obj2.id) &
#     F.like(obj1.type, 'vehicle%') &
#     F.like(obj2.type, 'vehicle%') &
#     F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) &
#     (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
#     (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
#     (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
#     (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
#     F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 50, 135) &
#     F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
#     (F.min_distance(cam.ego, F.road_segment('intersection')) < 10) &
#     F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
# )

In [ ]:
## Get all possible car1 + car2 instances
instances = []
for sample in nusc.sample:
  scene = nusc.get('scene', sample['scene_token'])
  log = nusc.get('log', scene['log_token'])
  if scene['name'] == 'scene-0757' and log['location'] == 'boston-seaport':
    if log['location'] == 'boston-seaport':
      for annotation1 in sample['anns']:
        for annotation2 in sample['anns']:
          annotation_metadata1 = nusc.get('sample_annotation', annotation1)
          annotation_metadata2 = nusc.get('sample_annotation', annotation2)
          if annotation_metadata1['category_name'] == 'vehicle.car' or annotation_metadata1['category_name'] == 'vehicle.bus.rigid' or annotation_metadata1['category_name'] == 'vehicle.truck':
            if annotation_metadata2['category_name'] == 'vehicle.car' or annotation_metadata1['category_name'] == 'vehicle.bus.rigid' or annotation_metadata2['category_name'] == 'vehicle.truck':
              CAM_FRONT_SENSOR = sample['data']['CAM_FRONT']
              sample_data = nusc.get('sample_data', CAM_FRONT_SENSOR)
              ego_pose_token = sample_data['ego_pose_token']
              instances.append(Instance(timestamp=sample_data['timestamp'], sample_record=sample, annotation_token=annotation1, annotation_token2=annotation2, ego_pose_token=ego_pose_token))
len(instances)

In [ ]:
## Filter for cars  that are within 50 meters of the ego
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation1 = nusc.get('sample_annotation', instance.annotation_token)
  annotation2 = nusc.get('sample_annotation', instance.annotation_token2)

  car1_trans = np.array(annotation1['translation'])
  car2_trans = np.array(annotation2['translation'])
  ego_trans = np.array(ego_pose['translation'])
  distance1 = np.linalg.norm(car1_trans - ego_trans)
  distance2 = np.linalg.norm(car2_trans - ego_trans)

  if distance1 < 50 and distance2 < 50:
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter so that cars are visible from ego
new_instances = []
for instance in instances:
  car1_visible = is_visible(sample_record=instance.sample_record, annotation_token=instance.annotation_token)
  car2_visible = is_visible(sample_record=instance.sample_record, annotation_token=instance.annotation_token2)
  if car1_visible and car2_visible:
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter for relative headings wrt to ego
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation1 = nusc.get('sample_annotation', instance.annotation_token)
  annotation2 = nusc.get('sample_annotation', instance.annotation_token2)

  
  ego_heading = get_heading(ego_pose['rotation'])
  car1_heading = get_heading(annotation1['rotation'])
  car2_heading = get_heading(annotation2['rotation'])
  
  car1_diff = normalizeAngle(ego_heading - car1_heading)
  car2_diff = normalizeAngle(ego_heading - car2_heading)
  
  if math.radians(50) < car1_diff and car1_diff < math.radians(135):
    if math.radians(-135) < car2_diff and car2_diff < math.radians(-50):
      if abs(car1_heading - car2_heading) > math.radians(100):
        new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter so that for cars that are in an intersection
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation1 = nusc.get('sample_annotation', instance.annotation_token)
  annotation2 = nusc.get('sample_annotation', instance.annotation_token2)

  car1_trans = np.array(annotation1['translation'])
  car2_trans = np.array(annotation2['translation'])
  ego_trans = np.array(ego_pose['translation'])
  
  if is_contained_intersection(car1_trans) and is_contained_intersection(car2_trans):
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter for egos whos heading is aligned with the road direction
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)

  ped_trans = np.array(annotation['translation'])
  ego_trans = np.array(ego_pose['translation'])
  
  ego_heading = get_heading(ego_pose['rotation'])
  road_direction = get_road_direction(ego_trans)
  if road_direction == None:
    new_instances.append(instance)
  elif normalizeAngle(abs(ego_heading - road_direction)) < math.radians(15):
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
results = {}
for instance in instances:
  camera_token = instance.sample_record['data']['CAM_FRONT']
  camera_data = nusc.get('sample_data', camera_token)
  results[camera_data['token']] = camera_data['filename'] # .split('/')[2]

In [ ]:
for token in results:
  print(results[token])
  nusc.render_sample_data(token)

In [ ]:
############# Validation ####################
expected_imgs = ['sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657111412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657111762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657112862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657114362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657114362404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657114612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657114612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657115012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657115112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657115162404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657116112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657117662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657117662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657117762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657117762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657117762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657117912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120512404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657120912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121162404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657121912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122512404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657122912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123512404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657123912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124112404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124112404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124112404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124512404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657124912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125112404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125512404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657125912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657126012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657126012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657126862404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127512404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657128012404.jpg']

expected_imgs = [x for x in expected_imgs if "samples" in x]

missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in results.values():
        missing += 1
        # missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
# show_images(missing_images, sample=10)

extra = 0

extra_images = {}
for img in results:
    if results[img] not in expected_imgs:
        extra += 1
        # extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(results), "=", 100 * extra / len(results), "%")
# show_images(extra_images.keys(), sample=10)

In [ ]:
print(list(results.values()))

In [ ]:
fig, ax = nusc_map.render_layers(nusc_map.non_geometric_layers, figsize=1)

In [ ]:
nusc.list_scenes()

## Figure 14


In [ ]:
# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

In [ ]:
# world = world.filter("lambda obj1, cam: " + 
#         "F.like(obj1.object_type, 'vehicle%') and " +
#         "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
#         "F.view_angle(obj1, cam.ego, cam.timestamp) < 70 / 2 and " +
#         "F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp), -180, -90) and " +
#         "F.contained(cam.ego, F.road_segment('road')) and " +
#         "F.contained(obj1.traj, F.road_segment('road'), cam.timestamp) and " +
#         "F.angle_between(F.facing_relative(obj1, F.road_direction(obj1.traj, cam.timestamp, cam.ego), cam.timestamp), -15, 15) and " +
#         "F.distance(cam.ego, obj1, cam.timestamp) < 10"
# )

In [ ]:
## Get all possible car + ego instances
instances = []
for sample in nusc.sample:
  scene = nusc.get('scene', sample['scene_token'])
  log = nusc.get('log', scene['log_token'])
  if log['location'] == 'boston-seaport' and scene["name"] == "scene-0769":
    for annotation in sample['anns']:
      annotation_metadata = nusc.get('sample_annotation', annotation)
      if annotation_metadata['category_name'] == 'vehicle.car' or annotation_metadata['category_name'] == 'vehicle.bus.rigid' or annotation_metadata['category_name'] == 'vehicle.truck':
        CAM_FRONT_SENSOR = sample['data']['CAM_FRONT']
        sample_data = nusc.get('sample_data', CAM_FRONT_SENSOR)
        ego_pose_token = sample_data['ego_pose_token']
        instances.append(Instance(timestamp=sample_data['timestamp'], sample_record=sample, annotation_token=annotation, ego_pose_token=ego_pose_token))
len(instances)

In [ ]:
## Filter for cars  that are within 10 meters of the ego
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation1 = nusc.get('sample_annotation', instance.annotation_token)

  car1_trans = np.array(annotation1['translation'])
  ego_trans = np.array(ego_pose['translation'])
  distance1 = np.linalg.norm(car1_trans - ego_trans)

  if distance1 < 50:
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter so that cars are visible from ego
new_instances = []
for instance in instances:
  car1_visible = is_visible(sample_record=instance.sample_record, annotation_token=instance.annotation_token)
  if car1_visible:
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter for cars whos heading is aligned with the road direction
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)

  car_trans = np.array(annotation['translation'])
  ego_trans = np.array(ego_pose['translation'])
  
  
  car_heading = get_heading(annotation['rotation'])
  road_direction = get_road_direction(car_trans)
  if road_direction == None:
    new_instances.append(instance)
  elif math.radians(-15) < normalizeAngle(abs(car_heading - road_direction)) and normalizeAngle(abs(car_heading - road_direction)) < math.radians(15):
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
## Filter for egos who are moving in opposite direction
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)
  
  ego_trans = np.array(ego_pose['translation'])

  road_direction = get_road_direction(ego_trans)
  ego_heading = get_heading(ego_pose['rotation'])
  if road_direction == None:
    new_instances.append(instance)
    continue
  diff = normalizeAngle(ego_heading - road_direction) 
  if (math.radians(-180) < diff and diff < math.radians(-90)) or (math.radians(90) < diff and diff < math.radians(180)):
    new_instances.append(instance)
instances = new_instances
len(instances)

In [ ]:
results = {}
for instance in instances:
  camera_token = instance.sample_record['data']['CAM_FRONT']
  camera_data = nusc.get('sample_data', camera_token)
  results[camera_data['token']] = camera_data['filename'] #.split('/')[2]

In [ ]:
for token in results:
  print(results[token])
  nusc.render_sample_data(token)

In [ ]:
############# Validation ####################
expected_imgs = ['sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657490912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491512404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491612404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491612404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491662404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491862404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492012404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492112404.jpg',
 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492412404.jpg']

expected_imgs = list(set([x for x in expected_imgs if "samples" in x]))

missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in results.values():
        missing += 1
        # missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
# show_images(missing_images, sample=10)

extra = 0
extra_images = []
for img in results:
    if results[img] not in expected_imgs:
        extra += 1
        extra_images.append(results[img])
print("Percentage of images in query but not in expected results: ", extra, "/", len(results), "=", 100 * extra / len(results), "%")
# show_images(extra_images.keys(), sample=10)

In [ ]:
print(results.values())
print(expected_imgs)

In [ ]:
print('samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491612404.jpg' in results.values())

In [ ]:
extra_images

In [ ]:
print(list(results.values()))

In [ ]:
fig, ax = nusc_map.render_layers(nusc_map.non_geometric_layers, figsize=1)

In [ ]:
nusc.list_scenes()

## Figure 15


In [ ]:
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# point1 = OrientedPoint ahead of ego by Range(0, 40)
# Car at (point1 offset by Range(-1, 1) @ 0),
#     facing Range(-15, 15) deg relative to roadDirection

In [ ]:
# world = world.filter(
#     (F.like(car1.type, 'car') | F.like(car1.type, 'truck')) &
#     (F.like(car2.type, 'car') | F.like(car2.type, 'truck')) &
#     (F.like(opposite_car.type, 'car') | F.like(opposite_car.type, 'truck')) &
#     (opposite_car.id != car2.id) &
#     (car1.id != car2.id) &
#     (car1.id != opposite_car.id) &

#     F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) &
#     (F.view_angle(car1.traj@cam.time, cam.ego) < 70 / 2) &
#     (F.distance(cam.ego, car1.traj@cam.time) < 50) &
# #     F.angle_between(F.facing_relative(car1.traj@cam.time, cam.ego), -15, 15) &
# #     F.angle_between(F.facing_relative(car1.traj@cam.time, F.road_direction(car1.traj@cam.time, cam.ego)), -15, 15) &
#     F.ahead(car1.traj@cam.time, cam.ego) &
#     F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) &
#     (F.convert_camera(opposite_car.traj@cam.time, cam.ego) > [-10, 0]) &
#     (F.convert_camera(opposite_car.traj@cam.time, cam.ego) < [-1, 50]) &
#     F.angle_between(F.facing_relative(opposite_car.traj@cam.time, cam.ego), 140, 180) &
#     (F.distance(opposite_car@cam.time, car2@cam.time) < 40) &
# #     F.angle_between(F.facing_relative(car2.traj@cam.time, F.road_direction(car2.traj@cam.time, cam.ego)), -15, 15) &
#     F.ahead(car2.traj@cam.time, opposite_car.traj@cam.time)
# )

In [41]:
## Get all possible car + car + car + ego instances
instances = []
for sample in nusc.sample:
  scene = nusc.get('scene', sample['scene_token'])
  log = nusc.get('log', scene['log_token'])
  if log['location'] == 'boston-seaport' and scene["name"] == "scene-0207":
    for annotation1 in sample['anns']:
      for annotation2 in sample['anns']:
        for annotation3 in sample['anns']:
          annotation1_metadata = nusc.get('sample_annotation', annotation1)
          annotation2_metadata = nusc.get('sample_annotation', annotation2)
          annotation3_metadata = nusc.get('sample_annotation', annotation3)
          if annotation1 != annotation2 and annotation2 != annotation3 and annotation1 != annotation3:
            if (annotation1_metadata['category_name'] == 'vehicle.car' or annotation1_metadata['category_name'] == 'vehicle.truck') \
              and (annotation2_metadata['category_name'] == 'vehicle.car' or annotation2_metadata['category_name'] == 'vehicle.truck') \
              and (annotation3_metadata['category_name'] == 'vehicle.car' or annotation3_metadata['category_name'] == 'vehicle.truck'):
              CAM_FRONT_SENSOR = sample['data']['CAM_FRONT']
              sample_data = nusc.get('sample_data', CAM_FRONT_SENSOR)
              ego_pose_token = sample_data['ego_pose_token']
              instances.append(Instance(timestamp=sample_data['timestamp'], sample_record=sample, annotation_token=annotation1, ego_pose_token=ego_pose_token, annotation_token2=annotation2, annotation_token3=annotation3))
len(instances)

260334

In [43]:
## Filter for distances
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation1 = nusc.get('sample_annotation', instance.annotation_token)
  annotation2 = nusc.get('sample_annotation', instance.annotation_token2)
  annotation3 = nusc.get('sample_annotation', instance.annotation_token3)

  oppposite_car_trans = np.array(annotation3['translation'])
  car_2_trans = np.array(annotation2['translation'])
  car_1_trans = np.array(annotation1['translation'])
  ego_trans = np.array(ego_pose['translation'])
  distance1 = np.linalg.norm(oppposite_car_trans - car_2_trans)
  distance2 = np.linalg.norm(car_1_trans - ego_trans)

  if distance1 < 40 and distance2 < 50:
    new_instances.append(instance)
instances = new_instances
len(instances)

83488

In [44]:
## Filter so that car 1s are visible from ego
new_instances = []
for instance in instances:
  car1_visible = is_visible(sample_record=instance.sample_record, annotation_token=instance.annotation_token)
  car2_visible = is_visible(sample_record=instance.sample_record, annotation_token=instance.annotation_token2)
  car3_visible = is_visible(sample_record=instance.sample_record, annotation_token=instance.annotation_token3)

  if car1_visible and car2_visible and car3_visible:
    new_instances.append(instance)
instances = new_instances
len(instances)

2072

In [48]:
## Filter so that car1s that are ahead of ego
new_instances = []
for instance in instances:
  # convert_camera(cam_position, cam_heading, obj_point)
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation1 = nusc.get('sample_annotation', instance.annotation_token)
  car_1_trans = np.array(annotation1['translation'])
  ego_trans = np.array(ego_pose['translation'])
  ego_heading = get_heading(ego_pose['rotation'])
  
  conv_x, conv_y = convert_camera(ego_trans, ego_heading, car_1_trans)
  if conv_y > 0:
    new_instances.append(instance)
instances = new_instances
len(instances)

2008

In [49]:
## Filter so that car2s that are ahead of opposite_car
new_instances = []
for instance in instances:
  # convert_camera(cam_position, cam_heading, obj_point)
  annotation2 = nusc.get('sample_annotation', instance.annotation_token2)
  annotation3 = nusc.get('sample_annotation', instance.annotation_token3)
  car_2_trans = np.array(annotation2['translation'])
  opposite_car_trans = np.array(annotation3['translation'])
  opposite_car_heading = get_heading(annotation3['rotation'])
  
  conv_x, conv_y = convert_camera(opposite_car_trans, opposite_car_heading, car_2_trans)
  if conv_y > 0:
    new_instances.append(instance)
instances = new_instances
len(instances)

1208

In [59]:
t = instances

In [68]:
instances = t

In [69]:
## Filter for convertCameras
new_instances = []
for instance in instances:
  # convert_camera(cam_position, cam_heading, obj_point)
  annotation3 = nusc.get('sample_annotation', instance.annotation_token3)

  opposite_car_trans = np.array(annotation3['translation'])
  opposite_car_heading = get_heading(annotation3['rotation'])
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  ego_trans = np.array(ego_pose['translation'])
  ego_heading = get_heading(ego_pose['rotation'])
  
  conv_x, conv_y = convert_camera(ego_trans, ego_heading, opposite_car_trans)
  # print(conv_x, conv_y)
  if 1 < conv_y and conv_y < 10 and 0 < conv_x and conv_x < 50:
    new_instances.append(instance)
instances = new_instances
len(instances)

121

In [53]:
## Filter for egos whos heading is aligned with the road direction
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation = nusc.get('sample_annotation', instance.annotation_token)

  ped_trans = np.array(annotation['translation'])
  ego_trans = np.array(ego_pose['translation'])
  
  ego_heading = get_heading(ego_pose['rotation'])
  road_direction = get_road_direction(ego_trans)
  if road_direction == None:
    new_instances.append(instance)
  elif normalizeAngle(abs(ego_heading - road_direction)) < math.radians(15):
    new_instances.append(instance)
instances = new_instances
len(instances)

1208

In [23]:
temp = instances

In [28]:
instances = temp

In [54]:
## Filter for ego and opposite car headings
new_instances = []
for instance in instances:
  ego_pose = nusc.get('ego_pose', instance.ego_pose_token)
  annotation3 = nusc.get('sample_annotation', instance.annotation_token3)
  
  ego_heading = get_heading(ego_pose['rotation'])
  opposite_car_heading = get_heading(annotation3['rotation'])
  if math.radians(140) < abs(normalizeAngle(ego_heading - opposite_car_heading)) and abs(normalizeAngle(ego_heading - opposite_car_heading)) < math.radians(180):
    new_instances.append(instance)
instances = new_instances
len(instances)

565

In [73]:
results = {}
for instance in instances:
  camera_token = instance.sample_record['data']['CAM_FRONT']
  camera_data = nusc.get('sample_data', camera_token)
  results[camera_data['token']] = camera_data['filename'] #.split('/')[2]

In [74]:
for token in results:
  print(results[token])
  # nusc.render_sample_data(token)

samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621915112404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621915662404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621916162404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621916662404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621917162404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621918162404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621918662404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621919162404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621919662404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920662404.jpg
samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621921162404.jpg
samples/CAM_FRONT/n008-2018-

In [72]:
############# Validation ####################
expected_imgs = ['sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621918762404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920012404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920112404.jpg',
 'samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920362404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920512404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920612404.jpg',
 'samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621920662404.jpg',
 'samples/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621923112404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621923162404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621923262404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621923412404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621927912404.jpg',
 'sweeps/CAM_FRONT/n008-2018-07-26-12-13-50-0400__CAM_FRONT__1532621928262404.jpg']

expected_imgs = list(set([x for x in expected_imgs if "samples" in x]))

missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in results.values():
        missing += 1
        # missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
# show_images(missing_images, sample=10)

extra = 0
extra_images = {}
for img in results:
    if results[img] not in expected_imgs:
        extra += 1
        # extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(results), "=", 100 * extra / len(results), "%")
# show_images(extra_images.keys(), sample=10)

Percentage of expected results missing from query:  0 / 3 = 0.0 %
Percentage of images in query but not in expected results:  23 / 26 = 88.46153846153847 %


In [ ]:
print(expected_imgs)
print(list(results.values()))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from PIL import Image

RAW_DATA_DIR = "/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Trainval/"
# Create figure and axes

for img in expected_imgs:
    im = Image.open(RAW_DATA_DIR + img)
    # Display the image
    plt.figure()
    plt.imshow(im)
# plt.show()

In [ ]:
fig, ax = nusc_map.render_layers(nusc_map.non_geometric_layers, figsize=1)

In [ ]:
nusc.list_scenes()